In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import beta
import seaborn as sns
import geopandas as gpd

### URBAN RURAL

In [ ]:
urban = pd.read_csv("data/urban_rural.csv")
counts = urban['Urban_rural_flag'].value_counts()
urban_probs = counts / counts.sum()
p_urban = urban_probs['Urban']
urban_sample = np.random.binomial(n=1, p=p_urban, size=1)[0]
print(urban_sample)

1


### POPULATION DENSITY

In [29]:
popden = pd.read_csv("data/population_density.csv", dtype={"LAD2021": "string", "OA21CD": "string", "Total": "Int64"})
merged = popden.merge(urban, on="OA21CD", how="left")
popden_urbanrural = merged[['LAD2021','OA21CD','Total','Urban_rural_flag']]

flag = 'urban' if urban_sample == 1 else 'rural'
total  = popden_urbanrural[popden_urbanrural["Urban_rural_flag"].str.lower() == flag]["Total"]

log = np.log(total)

mu, sigma = log.mean(), log.std()

popden_sample = np.random.lognormal(mean=mu, sigma=sigma, size=1)[0]
print(popden_sample)

158.97244218707326


### DISABILITY RATE

In [99]:
disability = pd.read_csv("data/disabled.csv")
total_disability = disability.groupby('Disability (3 categories)')['Observation'].sum()
#disability_probs = total_disability / total_disability.sum()

alpha = total_disability['Disabled under the Equality Act'] + 1
beta = total_disability['Not disabled under the Equality Act'] + 1

disabled_sample = np.random.beta(alpha, beta, size=1)[0]
print(disabled_sample)


0.18282935194095662


### ENGLISH PROFICIENCY

In [ ]:
def map_proficiency(category):
    if category in [
        "Main language is English (English or Welsh in Wales)",
        "Main language is not English (English or Welsh in Wales): Can speak English very well or well"
    ]:
        return "Good English Proficiency"
    elif category == "Main language is not English (English or Welsh in Wales): Cannot speak English or cannot speak English well":
        return "Bad English Proficiency"
    else:
        return None

english = pd.read_csv("data/english_proficiency.csv")   
total_english = english.groupby('Proficiency in English language (4 categories)')['Observation'].sum()
english['Proficiency_Group'] = english['Proficiency in English language (4 categories)'].apply(map_proficiency)
grouped_english = english.groupby('Proficiency_Group')['Observation'].sum()
#english_probs = grouped_english / grouped_english.sum()

alpha = grouped_english['Good English Proficiency'] + 1
beta = grouped_english['Bad English Proficiency'] + 1

english_sample = np.random.beta(alpha, beta, size=1)[0]
print(english_sample)

2699192 66413
0.9761280833866088


### MEAN INCOME

In [4]:
from scipy.stats import skewnorm, lognorm

income = pd.read_csv("data/mean_income.csv")
income['Total annual income (£)'] = (
    income['Total annual income (£)']
    .str.strip()        
    .str.replace(',', '')      
    .astype(float)       
)

log_income = np.log(income['Total annual income (£)'])
shape, loc, scale = skewnorm.fit(log_income)
sample_log = skewnorm.rvs(shape, loc=loc, scale=scale, size=1)
income_sample = np.exp(sample_log)[0]
print(income_sample)

39535.37691370101


### LOW-INCOME FRACTION (NOT DONE)

In [5]:

median = income['Total annual income (£)'].median()
low_income_threshold = 0.6 * median
print(low_income_threshold)

variance_log_skewnorm = skewnorm.var(shape, loc=loc, scale=scale)

###### PART THAT DOESNT SEEM RIGHT
meanlog = np.log(income_sample)
std = np.sqrt(variance_log_skewnorm)
print(std)
prob_low_income = lognorm.cdf(low_income_threshold, s=std, scale=np.exp(meanlog))
print(prob_low_income)


23520.0
0.1792183634234001
0.0018787571088464375


### MEAN PROPERTY VALUE

In [160]:
property = pd.read_csv("data/property_value.csv")
property = property.dropna(subset=['price', 'property_type', 'duration'])
property = property[property['price'] > 0]

log_property = np.log(property['price'])
shape, loc, scale = skewnorm.fit(log_property)
sample_log = skewnorm.rvs(shape, loc=loc, scale=scale, size=1)
property_sample = np.exp(sample_log)[0]
print(property_sample)


172189.41832701166


### DISTANCE TO WATER (DENSITY)

In [247]:
from scipy.stats import gamma

gdf = gpd.read_file("data/watercourse_density/watercourse_density.shp")
length = gdf["length_km"]
pct_zero = (length == 0).sum() / len(length)
zero_sample = np.random.binomial(n=1, p=pct_zero, size=1)[0]

gdf_nonzero = gdf[gdf["length_km"] != 0].copy()
length_nonzero = gdf_nonzero["length_km"]
shape, loc, scale = gamma.fit(length_nonzero)
length_sample = 0 if zero_sample == 1 else gamma.rvs(a=shape, loc=loc, scale=scale, size=1)[0]
print(length_sample*1000) # metres


347.9868995474528


### DISTANCE TO WATER (DISTANCE) MIGHT HAVE TO CHOOSE ONE OR THE OTHER

In [ ]:
gdf = gpd.read_file("data/watercourse_distance/watercourse_distance.shp")
distance = gdf["distance_t"]
shape, loc, scale = gamma.fit(distance)

max_distance = distance.max()
epsilon = 0.01
scale_factor = max_distance / (length_sample + epsilon)

distance_sample = gamma.rvs(a=shape, loc=loc, scale=scale, size=10)
#if length_sample == 0:
#    distance_sample = max_distance - distance_sample
print(distance_sample) # metres



[ 648.2645334   125.83904867  482.52047465  623.35400269  437.18838751
   63.01059675 1787.34533752  957.83382439  200.07151216  502.90078147]
